In [ ]:
import numpy as np
from pinocchio.utils import eye, rand, zero, skew, cross, rotate
import pinocchio as pin

In [ ]:
# Position
R = eye(3); p = zero(3)
M0 = pin.SE3(R, p)
M = pin.SE3.Random()
M.translation = p; M.rotation = R
M

In [ ]:
# Velocities
v = zero(3); w = zero(3)
nu0 = pin.Motion(v, w)
nu = pin.Motion.Random()
nu.linear = v; nu.angular = w
nu

In [ ]:
# Forces
f = zero(3); tau = zero(3)
phi0 = pin.Force(f, tau)
phi = pin.Force.Random()
phi.linear = f; phi.angular = tau
phi

In [ ]:
from example_robot_data import load

In [ ]:
robot = load('panda')

from pinocchio.robot_wrapper import RobotWrapper
#URDF = 'ur5_gripper.urdf'
#URDF = '/opt/openrobots/share/ur5_description/urdf/ur5_gripper.urdf'
URDF = '/opt/anaconda3/envs/siro/share/example-robot-data/robots/ur_description/urdf/ur5_gripper.urdf'
robot = RobotWrapper.BuildFromURDF(URDF)


In [ ]:
# Get index of end effector
 
idx = 2#robot.index('wrist_3_joint')
q = zero(robot.nq)

# Compute and get the placement of joint number idx
 
placement = robot.placement(q, idx)
# Be carreful, Python always returns references to values.
# You can often .copy() the object to avoid side effects
# Only get the placement
placement = robot.data.oMi[idx].copy()
placement

In [ ]:
robot.initViewer(loadModel=True)

In [ ]:
from pinocchio.visualize import MeshcatVisualizer
import pinocchio as pin

In [ ]:

import sys
import os
from os.path import dirname, join, abspath
 
# Load the URDF model.
# Conversion with str seems to be necessary when executing this file with ipython
pinocchio_model_dir = '/opt/anaconda3/envs/siro/share/'
 
model_path = join(pinocchio_model_dir,"example-robot-data/robots")
mesh_dir = pinocchio_model_dir
urdf_filename = "talos_reduced.urdf"
urdf_model_path = join(join(model_path,"talos_data/robots"),urdf_filename)

#model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir, pin.JointModelFreeFlyer())

In [ ]:
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)

In [ ]:
# Start a new MeshCat server and client.
# Note: the server can also be started separately using the "meshcat-server" command in a terminal:
# this enables the server to remain active after the current script ends.
#
# Option open=True pens the visualizer.
# Note: the visualizer can also be opened seperately by visiting the provided URL.
try:
    viz.initViewer(open=False)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)

In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
 
# Load the robot in the viewer.
viz.loadViewerModel()

In [ ]:
qr = np.random.rand(9)
qr[-2:]=0
viz.display(qr)

In [ ]:
# Display a robot configuration.
q0 = pin.neutral(robot.model)
viz.display(q0)

viz.displayCollisions(True)
viz.displayVisuals(False)

In [ ]:
viz.displayCollisions(False)
viz.displayVisuals(True)



mesh = robot.visual_model.geometryObjects[0].geometry
mesh.buildConvexRepresentation(True)
convex = mesh.convex

if convex is not None:
    placement = pin.SE3.Identity()
    placement.translation[0] = 2.
    geometry = pin.GeometryObject("convex",0,convex,placement)
    geometry.meshColor = np.ones((4))
    robot.visual_model.addGeometryObject(geometry)

# Display another robot.
viz2 = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz2.initViewer(viz.viewer)
viz2.loadViewerModel(rootNodeName = "pinocchio2")
q = q0.copy()
q[0] = 1.0
viz2.display(q)

viz.clean()

In [ ]:
from roboticstoolbox.models.DH import Panda

In [ ]:
panda = Panda()

# Centers of masses of Panda not yet implemented in Robotic Toolbox
r_arr = np.array(
    [
        [3.875e-03, 2.081e-03, 0],
        [-3.141e-03, -2.872e-02, 3.495e-03],
        [2.7518e-02, 3.9252e-02, -6.6502e-02],
        [-5.317e-02, 1.04419e-01, 2.7454e-02],
        [-1.1953e-02, 4.1065e-02, -3.8437e-02],
        [6.0149e-02, -1.4117e-02, -1.0517e-02],
        [1.0517e-02, -4.252e-03, -4.5403e-02],
    ]
)

for ii, link in enumerate(panda.links):
    link.r = r_arr[ii, :]

In [ ]:
for ii, link in enumerate(panda.links):
    print(ii,link.I)

In [ ]:
link.r

In [ ]:
def link_2_params(link):
    m = link.m
    mc_x, mc_y, mc_z = link.r
    I_xx = link.I[0,0]
    I_xy = link.I[0,1]
    I_xz = link.I[0,2]
    I_yy = link.I[1,1]
    I_yz = link.I[1,2]
    I_zz = link.I[2,2]
    return np.array([m, mc_x, mc_y, mc_z, I_xx, I_xy, I_yy, I_xz, I_yz, I_zz])

In [ ]:
link.I

In [ ]:
_i1 = robot.model.inertias[0]

In [ ]:
for ii in range(7):
    robot.model.inertias[ii+1] = robot.model.inertias[ii+1].FromDynamicParameters(link_2_params(panda.links[ii]))

In [ ]:
_i1

In [ ]:
for name, inert in zip(robot.model.names, robot.model.inertias):
    print(name, inert)

In [ ]:
pin.forwardKinematics(robot.model, robot.data, qr)

In [ ]:
for name, oMi in zip(robot.model.names, robot.data.oMi):
    print(("{:<24} : {: .2f} {: .2f} {: .2f}"
          .format( name, *oMi.translation.T.flat )))

In [ ]:
robot.forwardKinematics(qr)

In [ ]:
qr

In [ ]:
v = zero(9)

In [ ]:
tau = zero(9)

In [ ]:
pin.crba(robot.model, robot.data, qr)

In [ ]:
pin.rnea(robot.model, robot.data, qr, v, tau)

In [ ]:
pin.aba(robot.model, robot.data, qr, v, tau)

In [ ]:
robot.data.J